## Grupo:
### Carlos Mauricio Arteaga Bermudez
### Miguel Angel Pablos Perea

In [174]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats # libreria estadistica de Scipy
from sklearn.feature_selection import RFE # RFE es para seleccionar modelos
from sklearn.model_selection import train_test_split # para dividir la base en train y test
from sklearn import linear_model # para modelo lineal
from sklearn.metrics import mean_squared_error, r2_score # para sacar las metricas
import statsmodels.api as sm  ## Parte estadistica
from statsmodels.sandbox.regression.predstd import wls_prediction_std  ## Parte estadistica

In [175]:
df = pd.read_csv('\\ean_est_python_2024\\Codigo\\Ejercicios\\student-mat.csv',sep=';')
df.head(5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


In [176]:
pd.set_option('display.max_columns', None)  # Muestra todas las columnas 
pd.set_option('display.max_rows', 50)  # Muestra todas las filas 


### Descripcion inicial de los datos

In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      395 non-null    object
 1   sex         395 non-null    object
 2   age         395 non-null    int64 
 3   address     395 non-null    object
 4   famsize     395 non-null    object
 5   Pstatus     395 non-null    object
 6   Medu        395 non-null    int64 
 7   Fedu        395 non-null    int64 
 8   Mjob        395 non-null    object
 9   Fjob        395 non-null    object
 10  reason      395 non-null    object
 11  guardian    395 non-null    object
 12  traveltime  395 non-null    int64 
 13  studytime   395 non-null    int64 
 14  failures    395 non-null    int64 
 15  schoolsup   395 non-null    object
 16  famsup      395 non-null    object
 17  paid        395 non-null    object
 18  activities  395 non-null    object
 19  nursery     395 non-null    object
 20  higher    

In [178]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


In [179]:
#Verificaion de datos nulos en el DataFrame
missing_values = df.isnull().sum()
missing_values

school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
G1            0
G2            0
G3            0
dtype: int64

### Identificando las variables numericas y categoricas para obtener los dummies.

In [180]:
col_categoricas = df.select_dtypes(include=['object']).columns
print(col_categoricas)

col_numericas = df.select_dtypes(exclude=['object']).columns
print(col_numericas)


Index(['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob',
       'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities',
       'nursery', 'higher', 'internet', 'romantic'],
      dtype='object')
Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2',
       'G3'],
      dtype='object')


In [181]:
df.corr(numeric_only=True)

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
age,1.000000,-0.163658,-0.163438,0.070641,-0.004140,0.243665,0.053940,0.016434,0.126964,0.131125,0.117276,-0.062187,0.175230,-0.064081,-0.143474,-0.161579
Medu,-0.163658,1.000000,0.623455,-0.171639,0.064944,-0.236680,-0.003914,0.030891,0.064094,0.019834,-0.047123,-0.046878,0.100285,0.205341,0.215527,0.217147
Fedu,-0.163438,0.623455,1.000000,-0.158194,-0.009175,-0.250408,-0.001370,-0.012846,0.043105,0.002386,-0.012631,0.014742,0.024473,0.190270,0.164893,0.152457
traveltime,0.070641,-0.171639,-0.158194,1.000000,-0.100909,0.092239,-0.016808,-0.017025,0.028540,0.138325,0.134116,0.007501,-0.012944,-0.093040,-0.153198,-0.117142
studytime,-0.004140,0.064944,-0.009175,-0.100909,1.000000,-0.173563,0.039731,-0.143198,-0.063904,-0.196019,-0.253785,-0.075616,-0.062700,0.160612,0.135880,0.097820
failures,0.243665,-0.236680,-0.250408,0.092239,-0.173563,1.000000,-0.044337,0.091987,0.124561,0.136047,0.141962,0.065827,0.063726,-0.354718,-0.355896,-0.360415
famrel,0.053940,-0.003914,-0.001370,-0.016808,0.039731,-0.044337,1.000000,0.150701,0.064568,-0.077594,-0.113397,0.094056,-0.044354,0.022168,-0.018281,0.051363
freetime,0.016434,0.030891,-0.012846,-0.017025,-0.143198,0.091987,0.150701,1.000000,0.285019,0.209001,0.147822,0.075733,-0.058078,0.012613,-0.013777,0.011307
goout,0.126964,0.064094,0.043105,0.028540,-0.063904,0.124561,0.064568,0.285019,1.000000,0.266994,0.420386,-0.009577,0.044302,-0.149104,-0.162250,-0.132791
Dalc,0.131125,0.019834,0.002386,0.138325,-0.196019,0.136047,-0.077594,0.209001,0.266994,1.000000,0.647544,0.077180,0.111908,-0.094159,-0.064120,-0.054660


- [Medu y Fedu] son dos columnas que hablan de la educacion del padre y de la madre, son las que tienen a simple vista una correlacion positiva y significativa de un analisis rapido de correlacion de las variables numericas del DaraFrame
- [Dalc, Walc ] las variables relacionadas con el consumo de alcohol tambien estan relacionadas entre si.

In [182]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False,handle_unknown='ignore')

encoded_datos_categoricos = encoder.fit_transform(df[col_categoricas]) #En una sola operacion hacer el .fit y el .transform 

encoded_datos_categoricos

array([[1., 0., 1., ..., 0., 1., 0.],
       [1., 0., 1., ..., 1., 1., 0.],
       [1., 0., 1., ..., 1., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 1., 1., 0.],
       [0., 1., 0., ..., 1., 1., 0.]])

In [183]:
df_datos_categoricos = pd.DataFrame(encoded_datos_categoricos, 
                                    columns=encoder.get_feature_names_out(col_categoricas) ) #encoder.get_feature_names_out(col_categoricas) para obtener los nombres de las columnas
df_datos_categoricos

,school_GP,school_MS,sex_F,sex_M,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,Pstatus_T,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,schoolsup_no,schoolsup_yes,famsup_no,famsup_yes,paid_no,paid_yes,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
1,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
3,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
391,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
392,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
393,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [184]:
df_datos_categoricos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   school_GP          395 non-null    float64
 1   school_MS          395 non-null    float64
 2   sex_F              395 non-null    float64
 3   sex_M              395 non-null    float64
 4   address_R          395 non-null    float64
 5   address_U          395 non-null    float64
 6   famsize_GT3        395 non-null    float64
 7   famsize_LE3        395 non-null    float64
 8   Pstatus_A          395 non-null    float64
 9   Pstatus_T          395 non-null    float64
 10  Mjob_at_home       395 non-null    float64
 11  Mjob_health        395 non-null    float64
 12  Mjob_other         395 non-null    float64
 13  Mjob_services      395 non-null    float64
 14  Mjob_teacher       395 non-null    float64
 15  Fjob_at_home       395 non-null    float64
 16  Fjob_health        395 non

-Pasamos de tener 33 columnas a un total de 43.
-Ahora concatenamos las variables numericas del primer df con las df_datos_categoricos 

In [185]:
## concatenado los dos datframes. 


df_datos_numericos = df.select_dtypes(include=['number'])

df_ecoded_data = pd.concat([df_datos_numericos.reset_index(drop=True),
                           df_datos_categoricos.reset_index(drop=True)],axis=1
                           )
df_ecoded_data.sample(5) #Dataframe despues de aplicar en OneHotEncoder

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,school_GP,school_MS,sex_F,sex_M,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,Pstatus_T,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,schoolsup_no,schoolsup_yes,famsup_no,famsup_yes,paid_no,paid_yes,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
380,18,4,4,1,2,0,3,2,4,1,4,2,4,15,14,14,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
39,15,2,2,1,1,0,4,3,1,1,1,2,8,14,13,13,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
381,18,2,1,2,1,0,4,4,3,1,3,5,5,7,6,7,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
167,16,4,2,1,2,0,4,2,3,1,1,3,0,14,15,16,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
359,18,1,1,2,3,0,5,3,2,1,1,4,0,18,16,16,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0


In [186]:
## Al concatenar los dos dataframes obtenemos un total de 59 columnas.
df_ecoded_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 59 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                395 non-null    int64  
 1   Medu               395 non-null    int64  
 2   Fedu               395 non-null    int64  
 3   traveltime         395 non-null    int64  
 4   studytime          395 non-null    int64  
 5   failures           395 non-null    int64  
 6   famrel             395 non-null    int64  
 7   freetime           395 non-null    int64  
 8   goout              395 non-null    int64  
 9   Dalc               395 non-null    int64  
 10  Walc               395 non-null    int64  
 11  health             395 non-null    int64  
 12  absences           395 non-null    int64  
 13  G1                 395 non-null    int64  
 14  G2                 395 non-null    int64  
 15  G3                 395 non-null    int64  
 16  school_GP          395 non

## Modelo de regresion para la variable dependiente 'G1'

In [187]:
#Modelo de regresion lineal para la variable dependiente G1 
import statsmodels.api as sm

# Ajustar el modelo de regresión lineal múltiple
X = df_ecoded_data.drop(columns=['G1','G2','G3']) #Cargamos las columnas que van a funcionar como variables independientes haciendole drop a la columna G1,G2 Y G3 
y = df_ecoded_data['G1']

# Añadir una constante al modelo (intercepto)
X = sm.add_constant(X)

# Crear un modelo OLS
modelo = sm.OLS(y, X)

# Ajustar el modelo
resultado = modelo.fit()

# Imprimir el resumen del modelo
print(resultado.summary())


                            OLS Regression Results                            
Dep. Variable:                     G1   R-squared:                       0.334
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                     4.562
Date:                Wed, 09 Oct 2024   Prob (F-statistic):           3.63e-15
Time:                        15:35:19   Log-Likelihood:                -953.63
No. Observations:                 395   AIC:                             1987.
Df Residuals:                     355   BIC:                             2146.
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 1.3212      0.34

## Variables que muestran importancia en el modelo: esta lista de variables tienen un P>|t| < 0.05
- ['studytime','goout','school_GP','sex_M','address_U', 'famsize_LE3','Pstatus_T','Mjob_health','Mjob_services',
- 'Fjob_teacher','reason_reputation','guardian_other','schoolsup_no','famsup_no','paid_no','paid_yes','activities_no','activities_yes',
- 'nursery_no','nursery_yes','higher_yes','internet_yes','romantic_no','romantic_yes']

- El valor Cond. No. 9.93e+16 sugiere problemas de multicolinealidad, lo que puede indicar redundancia entre algunas variables independientes.



### Modelo ajustado con las variables seleccionadas

In [188]:
#Modelo de regresion lineal para la variable dependiente G1 AJUSTADO
import statsmodels.api as sm

# Ajustar el modelo de regresión lineal múltiple
X = df_ecoded_data[['studytime','goout','school_GP','sex_M','address_U', 'famsize_LE3','Pstatus_T','Mjob_health','Mjob_services','Fjob_teacher','reason_reputation',
                    'guardian_other','schoolsup_no','famsup_no','paid_no','paid_yes','activities_no','activities_yes','nursery_no','nursery_yes','higher_yes','internet_yes',
                    'romantic_no','romantic_yes'
                    ]]
y = df_ecoded_data['G1']

# Añadir una constante al modelo (intercepto)
X = sm.add_constant(X)

# Crear un modelo OLS
modelo = sm.OLS(y, X)

# Ajustar el modelo
resultado = modelo.fit()

# Imprimir el resumen del modelo
print(resultado.summary())

                            OLS Regression Results                            
Dep. Variable:                     G1   R-squared:                       0.235
Model:                            OLS   Adj. R-squared:                  0.194
Method:                 Least Squares   F-statistic:                     5.747
Date:                Wed, 09 Oct 2024   Prob (F-statistic):           4.36e-13
Time:                        15:35:19   Log-Likelihood:                -980.94
No. Observations:                 395   AIC:                             2004.
Df Residuals:                     374   BIC:                             2087.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 1.5152      0.42

## Interpretacion:
- $R^2$ 0.235 indica solo que el 23.5% de la variabilidad en G1 es explicada por las variables independientes seleccionadas.
- Adj. R-squared 0.194, indica que variables utilizadas en el modelo pueden no estar aportando informacion relevante. 
- Durbin-Watson: 2.119, este valor cercano a 2 sugiere que no hay autocorrelación significativa en los residuos, lo cual es positivo para la validez del modelo.
- Para este modelo en particular se debe considerar utilizar la regresión Ridge o Lasso para reducir la multicolinealidad y encontrar las variables que mejor expliquen la variabilidad de G1.



### Validacion del modelo

In [189]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Separar la variable objetivo y las variables independientes del modelo ajustado
X = df_ecoded_data[['studytime','goout','school_GP','sex_M','address_U', 'famsize_LE3','Pstatus_T','Mjob_health','Mjob_services','Fjob_teacher','reason_reputation',
                    'guardian_other','schoolsup_no','famsup_no','paid_no','paid_yes','activities_no','activities_yes','nursery_no','nursery_yes','higher_yes','internet_yes','romantic_no','romantic_yes'
                    ]]
y = df_ecoded_data['G1']

# Dividir los datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular las métricas de evaluación
mse = mean_squared_error(y_test, y_pred,squared=True)
rmse = mean_squared_error(y_test, y_pred, squared=False)  
r2 = r2_score(y_test, y_pred)

print("Error cuadratico medio (MSE):", mse)
print("Raiz del error cuadratico medio (RMSE):", rmse)
print("R-squared (R^2) en el conjunto de prueba:", r2)

Error cuadratico medio (MSE): 11.903113298998825
Raiz del error cuadratico medio (RMSE): 3.4500888827679246
R-squared (R^2) en el conjunto de prueba: 0.12943174777279753


c:\Users\aluca\miniconda3\envs\mi_primer_ambiente\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\aluca\miniconda3\envs\mi_primer_ambiente\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Solo el 12.9% de la variabilidad en G1 es explicada por el modelo.
- Las predicciones se desvían en aproximadamente 3.45 unidades de los valores reales de G1

### Prediccion

In [190]:
nuevo_estudiante = pd.DataFrame({
    'studytime': [2],
    'goout': [3],
    'school_GP': [1],
    'sex_M': [1],
    'address_U': [1],
    'famsize_LE3': [1],
    'Pstatus_T': [0],
    'Mjob_health': [1],
    'Mjob_services': [0],
    'Fjob_teacher': [0],
    'reason_reputation': [1],
    'guardian_other': [0],
    'schoolsup_no': [1],
    'famsup_no': [0],
    'paid_no': [1],
    'paid_yes': [0],
    'activities_no': [1],
    'activities_yes': [0],
    'nursery_no': [0],
    'nursery_yes': [1],
    'higher_yes': [1],
    'internet_yes': [1],
    'romantic_no': [1],
    'romantic_yes': [0]
})

# predicción utilizando el modelo entrenado
prediccion_nota = model.predict(nuevo_estudiante)

print("Predicción de la nota G1:", prediccion_nota[0])

Predicción de la nota G1: 13.697462002205686


-----------------------------------------------------------------------------------------------------

## Modelo de regresion para la variable 'G2'

In [191]:
#Modelo de regresion lineal para la variable dependiente G2 
import statsmodels.api as sm

# Ajustar el modelo de regresión lineal múltiple
X = df_ecoded_data.drop(columns=['G2','G3']) #Cargamos las columnas que van a funcionar como variables independientes haciendole drop a la columna G1 que va a funcionar como variable dependiente
y = df_ecoded_data['G2']

# Añadir una constante al modelo (intercepto)
X = sm.add_constant(X)

# Crear un modelo OLS
modelo = sm.OLS(y, X)

# Ajustar el modelo
resultado = modelo.fit()

# Imprimir el resumen del modelo
print(resultado.summary())

                            OLS Regression Results                            
Dep. Variable:                     G2   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.748
Method:                 Least Squares   F-statistic:                     30.17
Date:                Wed, 09 Oct 2024   Prob (F-statistic):           8.22e-91
Time:                        15:35:19   Log-Likelihood:                -790.25
No. Observations:                 395   AIC:                             1663.
Df Residuals:                     354   BIC:                             1826.
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.4685      0.23

### Modelo ajustado con las variables seleccionadas

Variables con un P>|t| < 0.05 ['traveltime', 'G1', 'address_U', 'famsize_LE3','Pstatus_A','Mjob_other','Fjob_services','reason_other','schoolsup_yes','paid_yes','internet_yes', 'romantic_no']

In [192]:
#Modelo de regresion lineal para la variable dependiente G2 Ajustado 
import statsmodels.api as sm

# Ajustar el modelo de regresión lineal múltiple
X = df_ecoded_data[[ 'traveltime','G1', 'address_U', 'famsize_LE3','Pstatus_A','Mjob_other',
                    'Fjob_services','reason_other','schoolsup_yes','paid_yes','internet_yes', 'romantic_no']]
y = df_ecoded_data['G2']

# Añadir una constante al modelo (intercepto)
X = sm.add_constant(X)

# Crear un modelo OLS
modelo = sm.OLS(y, X)

# Ajustar el modelo
resultado = modelo.fit()

# Imprimir el resumen del modelo
print(resultado.summary())

                            OLS Regression Results                            
Dep. Variable:                     G2   R-squared:                       0.759
Model:                            OLS   Adj. R-squared:                  0.751
Method:                 Least Squares   F-statistic:                     100.2
Date:                Wed, 09 Oct 2024   Prob (F-statistic):          5.90e-110
Time:                        15:35:19   Log-Likelihood:                -802.35
No. Observations:                 395   AIC:                             1631.
Df Residuals:                     382   BIC:                             1682.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -1.3543      0.557     -2.431

### Interpretacion:

- El valor $R^2$ 0.759 indica que el modelo explica el 75.9% lo que muestra un ajuste bueno.
- El valor de Adj. R-squared es ligeramente menor pero es acceptable e indica que el modelo no esta sobreajustado.
- El valor F-statistic (100.2) y un p_valor (5.90e-110) pequeño indican que el modelo es significativo y que las variables seleccionadas tienen un buen efecto predictivo.
- G1 es la variable independiente mas importante, cada aumento en G1 tiene un impacto casi lineal en G2. esto se intuye ya que G1 y G2 representan calificaciones de periodos sucesivos y están fuertemente correlacionadas.
- Mjob_other, tener una madre con otro tipo de trabajo se asocia positivamente con G2.
- reason_other, Si la razón para escoger la escuela es otra (no académica, cercana al hogar, etc.), esto tiene un impacto positivo en G2.
- schoolsup_yes, recibir apoyo escolar está asociado positivamente con G2, lo cual podría sugerir que el apoyo adicional ayuda a mejorar el rendimiento.
- paid_yes, Recibir clases pagadas está relacionado positivamente con G2.
- romantic_no, No estar en una relación romántica está asociado con una mejor calificación en G2.
- Durbin-Watson: 1.584 (2 ideal), indica que no hay una fuerte autocorrelación en los residuos del modelo.
- Los valores de Omnibus y Jarque-Bera muestran que los residuos no están distribuidos normalmente (p-valor < 0.001).
- La asimetría (skew) negativa y la alta curtosis indican que los residuos tienen una distribución sesgada con colas largas. (Puede que este modelo no capture completamente la variabilidad de G2)

- Seria buena idea utilizar un modelo de regresion mas selectivo como Ridge o Lasso, realizar un analisis de componentes o aplicar un analisis de factor de inflación de la varianza.


### Validacion del modelo

In [193]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Separar la variable objetivo y las variables independientes del modelo ajustado
X = df_ecoded_data[[ 'traveltime','G1', 'address_U', 'famsize_LE3','Pstatus_A','Mjob_other',
                    'Fjob_services','reason_other','schoolsup_yes','paid_yes','internet_yes', 'romantic_no']]
y = df_ecoded_data['G2']

# Dividir los datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular las métricas de evaluación
mse = mean_squared_error(y_test, y_pred,squared=True)
rmse = mean_squared_error(y_test, y_pred, squared=False)  
r2 = r2_score(y_test, y_pred)

print("Error cuadratico medio (MSE):", mse)
print("Raiz del error cuadratico medio (RMSE):", rmse)
print("R-squared (R^2) en el conjunto de prueba:", r2)

Error cuadratico medio (MSE): 2.6219243524813156
Raiz del error cuadratico medio (RMSE): 1.6192357309796852
R-squared (R^2) en el conjunto de prueba: 0.8158639987865338


c:\Users\aluca\miniconda3\envs\mi_primer_ambiente\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\aluca\miniconda3\envs\mi_primer_ambiente\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- El $R^2$ explica aproximadamente el 81.5% de la variabilidad en G2.
- Las predicciones tendran un error de 1.61 puntos de la calificacion. 

### Prediccion:

In [194]:
nuevo_estudiante = pd.DataFrame({
    'traveltime': [1],
    'G1': [15],
    'address_U': [1],
    'famsize_LE3': [0],
    'Pstatus_A': [1],
    'Mjob_other': [0],
    'Fjob_services': [1],
    'reason_other': [0],
    'schoolsup_yes': [1],
    'paid_yes': [0],
    'internet_yes': [1],
    'romantic_no': [1]
})

prediccion_G2 = model.predict(nuevo_estudiante)
print("Predicción de G2 para el nuevo estudiante:", prediccion_G2[0])

Predicción de G2 para el nuevo estudiante: 16.172592823859958


------------------------------------------------------------------------------------------------

## Modelo de regresion para la variable 'G3'

In [195]:
#Modelo de regresion lineal para la variable dependiente G3
import statsmodels.api as sm

# Ajustar el modelo de regresión lineal múltiple
X = df_ecoded_data.drop(columns=['G3']) #Cargamos las columnas que van a funcionar como variables independientes haciendole drop a la columna G1 que va a funcionar como variable dependiente
y = df_ecoded_data['G3']

# Añadir una constante al modelo (intercepto)
X = sm.add_constant(X)

# Crear un modelo OLS
modelo = sm.OLS(y, X)

# Ajustar el modelo
resultado = modelo.fit()

# Imprimir el resumen del modelo
print(resultado.summary())

                            OLS Regression Results                            
Dep. Variable:                     G3   R-squared:                       0.846
Model:                            OLS   Adj. R-squared:                  0.828
Method:                 Least Squares   F-statistic:                     47.21
Date:                Wed, 09 Oct 2024   Prob (F-statistic):          7.20e-119
Time:                        15:35:19   Log-Likelihood:                -791.99
No. Observations:                 395   AIC:                             1668.
Df Residuals:                     353   BIC:                             1835.
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.0860      0.23

### Modelo ajustado con las variables seleccionadas

- ['famrel', 'absences', 'G1', 'G2'] Variables con un P>|t| < 0.05

In [196]:
#Modelo de regresion lineal para la variable dependiente G3 Ajustado con las nuevas variables.
import statsmodels.api as sm

# Ajustar el modelo de regresión lineal múltiple
X = df_ecoded_data[['famrel', 'absences', 'G1', 'G2']] 
y = df_ecoded_data['G3']

# Añadir una constante al modelo (intercepto)
X = sm.add_constant(X)

# Crear un modelo OLS
modelo = sm.OLS(y, X)

# Ajustar el modelo
resultado = modelo.fit()

# Imprimir el resumen del modelo
print(resultado.summary())

                            OLS Regression Results                            
Dep. Variable:                     G3   R-squared:                       0.831
Model:                            OLS   Adj. R-squared:                  0.829
Method:                 Least Squares   F-statistic:                     478.2
Date:                Wed, 09 Oct 2024   Prob (F-statistic):          6.67e-149
Time:                        15:35:19   Log-Likelihood:                -810.46
No. Observations:                 395   AIC:                             1631.
Df Residuals:                     390   BIC:                             1651.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.4092      0.538     -6.342      0.0

- El $R^2$ 0.831 explica el 83.1% de la variabilidad en G3. 
- Adj. R-squared 0.829 muestra un ajuste robusto e indica que el modelo no esta sobreajustado. 
- F-statistic (478.2) alto y un p-valor (6.67e-149) muy bajo muestra un ajuste bueno y que el modelo es significativo para explicar G3
- La variable famrel, tiene una asociacion positiva lo que puede indicar que un aumento en la relacion familiar incrementa la calificaion de G3 en 0.342 puntos.
- La variable absences esta asociada positivamente con G3, lo que puede indicar un contexto mas profundo para su interpretacion.
- La calificación en G1 tiene una influencia significativa y positiva en G3
- Por cada punto adicional en G2, G3 aumenta en casi 1 punto. La fuerza de G2 como predictor de G3 se debe a su alta correlación y proximidad en el tiempo. 
- Durbin-Watson 1.829, muy cercano a 2 indica que no hay problemas graves de autocorrelación en los residuos.
- Los valores de las pruebas de Omnibus y Jarque-Bera sugieren que los residuos no están distribuidos normalmente (p < 0.05).
- La asimetría negativa (skew de -2.485) y la alta curtosis (11.032) indican que los residuos tienen una distribución con colas largas y sesgada a la izquierda.
- Cond. No. Un valor de condición de 100 indica que no hay un problema grave de multicolinealidad entre las variables independientes.

### Validacion del modelo:

In [197]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Separar la variable objetivo y las variables independientes del modelo ajustado
X = df_ecoded_data[['famrel', 'absences', 'G1', 'G2']]
y = df_ecoded_data['G3']

# Dividir los datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular las métricas de evaluación
mse = mean_squared_error(y_test, y_pred,squared=True)
rmse = mean_squared_error(y_test, y_pred, squared=False)  
r2 = r2_score(y_test, y_pred)

print("Error cuadratico medio (MSE):", mse)
print("Raiz del error cuadratico medio (RMSE):", rmse)
print("R-squared (R^2) en el conjunto de prueba:", r2)

Error cuadratico medio (MSE): 4.163105697228049
Raiz del error cuadratico medio (RMSE): 2.0403690100636327
R-squared (R^2) en el conjunto de prueba: 0.7969716605476178


c:\Users\aluca\miniconda3\envs\mi_primer_ambiente\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\aluca\miniconda3\envs\mi_primer_ambiente\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- La raiz del error cuadratico medio (RMSE): 2.04 nos indica que en promedio las predicciones sobre las notas van a estar desviadas 2.04 de la nota real  
- El valor R-squared $R^2$ explica al rededor del 79.7% de la variabilidad en G3.  

### Prediccion:

In [198]:
nuevo_estudiante = pd.DataFrame({
    'famrel': [4],
    'absences': [3],
    'G1': [15],
    'G2': [16]
})

prediccion_G3 = model.predict(nuevo_estudiante)
print("Predicción de G3 para el nuevo estudiante:", prediccion_G3[0])

Predicción de G3 para el nuevo estudiante: 16.207326442975457


-------------------------------------------------------------------------------------------

## Modelo de regresion logistica

- Agregamos una nueva columna a df_ecoded_data para indicar si el estudiante aprobo o no segun la condicion de que G1+G2+G3 > 40. Aprobo = 1, No aprobo = 0

In [199]:
df_ecoded_data

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,school_GP,school_MS,sex_F,sex_M,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,Pstatus_T,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,schoolsup_no,schoolsup_yes,famsup_no,famsup_yes,paid_no,paid_yes,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
0,18,4,4,2,2,0,4,3,4,1,1,3,6,5,6,6,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
1,17,1,1,1,2,0,5,3,3,1,1,3,4,5,5,6,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,15,1,1,1,2,3,4,3,2,2,3,3,10,7,8,10,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
3,15,4,2,1,3,0,3,2,2,1,1,5,2,15,14,15,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,16,3,3,1,2,0,4,3,2,1,2,5,4,6,10,10,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,20,2,2,1,2,2,5,5,4,4,5,4,11,9,9,9,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
391,17,3,1,2,1,0,2,4,5,3,4,2,3,14,16,16,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
392,21,1,1,1,1,3,5,5,3,3,3,3,3,10,8,7,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
393,18,3,2,3,1,0,4,4,1,3,4,5,0,11,12,10,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [200]:
#Generamos una columna adicional de nombre aprobado donde si se cumple la condicion que el puntaje del estudiante es mayor a 40 aprueba = True No aprueba = False. 
df_ecoded_data['aprobado']=(df_ecoded_data['G1']+df_ecoded_data['G2'] + df_ecoded_data['G3'] > 40).astype(int) #Utilizamos la funcion .astype(int) para converitir los valores booleanos en 1 y 0
df_ecoded_data

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,school_GP,school_MS,sex_F,sex_M,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,Pstatus_T,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,schoolsup_no,schoolsup_yes,famsup_no,famsup_yes,paid_no,paid_yes,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes,aprobado
0,18,4,4,2,2,0,4,3,4,1,1,3,6,5,6,6,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0
1,17,1,1,1,2,0,5,3,3,1,1,3,4,5,5,6,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0
2,15,1,1,1,2,3,4,3,2,2,3,3,10,7,8,10,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0
3,15,4,2,1,3,0,3,2,2,1,1,5,2,15,14,15,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1
4,16,3,3,1,2,0,4,3,2,1,2,5,4,6,10,10,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,20,2,2,1,2,2,5,5,4,4,5,4,11,9,9,9,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0
391,17,3,1,2,1,0,2,4,5,3,4,2,3,14,16,16,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1
392,21,1,1,1,1,3,5,5,3,3,3,3,3,10,8,7,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0
393,18,3,2,3,1,0,4,4,1,3,4,5,0,11,12,10,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0


In [201]:
df_ecoded_data['aprobado'].value_counts() 

aprobado
0    302
1     93
Name: count, dtype: int64

### Modelo de regresion logistica utilizando todas las variables del DF (obviamos las variables objetivo ['G1','G2','G3','aprobado'])

In [202]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

X = df_ecoded_data.drop(columns=['G1','G2','G3','aprobado'])
y = df_ecoded_data['aprobado']

#Dividimos el DataFrame en 20% para testeo y 80% para entrenar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creamos y entrenamos el modelo
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = log_reg.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo:", accuracy)

# Matriz de confusión y reporte de clasificación
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))


Precisión del modelo: 0.7215189873417721
Matriz de confusión:
[[49  6]
 [16  8]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.75      0.89      0.82        55
           1       0.57      0.33      0.42        24

    accuracy                           0.72        79
   macro avg       0.66      0.61      0.62        79
weighted avg       0.70      0.72      0.70        79



c:\Users\aluca\miniconda3\envs\mi_primer_ambiente\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


- La precisión del modelo (0.72) es de aproximadamente el (72.15%), lo cual indica que el modelo clasifica correctamente el estado de aprobación del estudiante en el 72% de los casos.
- la matriz de confucion muestra 49 verdaderos negativos (TN): Estudiantes que no aprueban y fueron clasificados correctamente, 8 falsos positivos (FP): Estudiantes que no aprueban, pero el modelo predijo que sí, 16 falsos negativos (FN): Estudiantes que aprueban, pero el modelo predijo que no, 6 verdaderos positivos (TP): Estudiantes que aprueban y fueron clasificados correctamente.
- El modelo esta mejor calibrado para identificar los estudiantes que no aprueban = 0 y tiene mas errores en clasificar los estudiantes que aprueban = 1


### Clase 0
- con un 75% de presicion el modelo tiene una buena tasa para identificar los estudiantes que no aprueban el curso.
- con un 0.89 de recall el modelo es efectivo capturando la mayoria de estudiantes que no aprueban. 
- con un 0.82 de f1-score se indica un buen balance para precisión y recall.

### Clase 1 
- con un 57% de presicion el modelo tiene una tasa baja de aciertos para estudiantes que aprueban. 
- con un 0.33 de recall el modelo es solo efectivo en el 33% de los estdiantes que aprueban.
- con un f1-score de 0.42 el modelo tiene dificultades para clasificar correctamente a los estudiantes que aprueban.

In [203]:
X.columns

Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'school_GP',
       'school_MS', 'sex_F', 'sex_M', 'address_R', 'address_U', 'famsize_GT3',
       'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Mjob_at_home', 'Mjob_health',
       'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home',
       'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher',
       'reason_course', 'reason_home', 'reason_other', 'reason_reputation',
       'guardian_father', 'guardian_mother', 'guardian_other', 'schoolsup_no',
       'schoolsup_yes', 'famsup_no', 'famsup_yes', 'paid_no', 'paid_yes',
       'activities_no', 'activities_yes', 'nursery_no', 'nursery_yes',
       'higher_no', 'higher_yes', 'internet_no', 'internet_yes', 'romantic_no',
       'romantic_yes'],
      dtype='object')

### Prediccion:

In [204]:
nueva_muestra = pd.DataFrame({
    'age': [16],
    'Medu': [4],
    'Fedu': [4],
    'traveltime': [1],
    'studytime': [2],
    'failures': [0],
    'famrel': [4],
    'freetime': [3],
    'goout': [3],
    'Dalc': [1],
    'Walc': [1],
    'health': [4],
    'absences': [2],
    'school_GP': [1],
    'school_MS': [0],
    'sex_F': [1],
    'sex_M': [0],
    'address_R': [0],
    'address_U': [1],
    'famsize_GT3': [1],
    'famsize_LE3': [0],
    'Pstatus_A': [0],
    'Pstatus_T': [1],
    'Mjob_at_home': [0],
    'Mjob_health': [1],
    'Mjob_other': [0],
    'Mjob_services': [0],
    'Mjob_teacher': [0],
    'Fjob_at_home': [0],
    'Fjob_health': [0],
    'Fjob_other': [1],
    'Fjob_services': [0],
    'Fjob_teacher': [0],
    'reason_course': [0],
    'reason_home': [0],
    'reason_other': [1],
    'reason_reputation': [0],
    'guardian_father': [0],
    'guardian_mother': [1],
    'guardian_other': [0],
    'schoolsup_no': [1],
    'schoolsup_yes': [0],
    'famsup_no': [1],
    'famsup_yes': [0],
    'paid_no': [1],
    'paid_yes': [0],
    'activities_no': [1],
    'activities_yes': [0],
    'nursery_no': [0],
    'nursery_yes': [1],
    'higher_no': [0],
    'higher_yes': [1],
    'internet_no': [0],
    'internet_yes': [1],
    'romantic_no': [1],
    'romantic_yes': [0]
})

# Realizar la predicción con el modelo entrenado
prediccion = log_reg.predict(nueva_muestra)

# Interpretar el resultado de la predicción
if prediccion[0] == 1:
    print("Predicción: El estudiante aprobará.")
else:
    print("Predicción: El estudiante no aprobará.")

Predicción: El estudiante no aprobará.


### Ajustando las variables seleccionadas. 

In [205]:
X = df_ecoded_data[['studytime', 'failures', 'famrel', 'absences', 'higher_yes', 'paid_yes', 'internet_yes', 'romantic_no']] 
y = df_ecoded_data['aprobado']

#Dividimos el DataFrame en 20% para testeo y 80% para prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creamos y entrenamos el modelo
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = log_reg.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo:", accuracy)

# Matriz de confusión y reporte de clasificación
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))

Precisión del modelo: 0.6835443037974683
Matriz de confusión:
[[54  1]
 [24  0]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.69      0.98      0.81        55
           1       0.00      0.00      0.00        24

    accuracy                           0.68        79
   macro avg       0.35      0.49      0.41        79
weighted avg       0.48      0.68      0.57        79



- La precisión del modelo (0.683) es de aproximadamente el (68.3%), lo cual indica que el modelo clasifica correctamente el estado de aprobación del estudiante en el 72% de los casos.
- la matriz de confucion muestra 54 verdaderos negativos (TN): Estudiantes que no aprueban y fueron clasificados correctamente, 1 falso positivo (FP): Estudiante que no aprueba, pero el modelo predijo que sí, 24 falsos negativos (FN): Estudiantes que aprueban, pero el modelo predijo que no, 0 verdaderos positivos (TP): No se logró clasificar correctamente a ningún estudiante que aprueba.
- El modelo tiene problemas para capturar la clase 1, lo que indica que esta altamente sesgado hacia la clase 0.


### Clase 0
- precision con un 0.69 quiere decir que el modelo predice un estudiante que no aprueba el 69% de las veces.
- un recall de 0.98 indica que el modelo es muy efectivo en capturar la mayoría de los estudiantes que no aprueban.
- f1-score: 0.81. Esto indica un buen balance entre precisión y exhaustividad para la clase 0.

### Clase 1 
- Todos los valores (precisión, recall y F1-score) son 0.0, lo que significa que el modelo no está capturando ningún estudiante que apruebe.

In [206]:
# Con statsmodel

X = df_ecoded_data[['studytime', 'failures', 'famrel', 'absences', 'higher_yes', 'paid_yes', 'internet_yes', 'romantic_no']]
y = df_ecoded_data['aprobado']

# Añadir una constante al modelo (intercepto)
X = sm.add_constant(X)

# Crear el modelo de regresión logística
log_reg = sm.Logit(y, X)

# Ajustar el modelo
resultado = log_reg.fit(maxiter=1000)

# Mostrar el resumen del modelo
print(resultado.summary())

         Current function value: 0.488460
         Iterations: 1000
                           Logit Regression Results                           
Dep. Variable:               aprobado   No. Observations:                  395
Model:                          Logit   Df Residuals:                      386
Method:                           MLE   Df Model:                            8
Date:                Wed, 09 Oct 2024   Pseudo R-squ.:                  0.1050
Time:                        15:35:19   Log-Likelihood:                -192.94
converged:                      False   LL-Null:                       -215.58
Covariance Type:            nonrobust   LLR p-value:                 3.264e-07
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -26.7698   1.32e+05     -0.000      1.000   -2.58e+05    2.58e+05
studytime        0.1761      0.145      1.217      0.224 

c:\Users\aluca\miniconda3\envs\mi_primer_ambiente\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


- Pseudo R-squ. el modelo solo puede explicar el 10.5% de la variabilidad de aprobado
- se recomienda hacer un proceso de escalamiento y normalizacion de variables.
- El modelo necesita ajustes para ser utilizado para predecir si un estudiante aprobara o no. 